In [3]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    # other params...
)

In [7]:
response  = llm.invoke("Capital of Sri lanka")
print(response.content)

<think>

</think>

The capital of Sri Lanka is **Sri Jayawardenepura Kotte**, which is the administrative capital. However, the largest city and commercial capital is **Colombo**.


In [42]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://amazon.jobs/en/jobs/3055226/software-engineer-i")
content1 = loader.load().pop().page_content
print(content1)

Software Engineer I - Job ID: 3055226 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsInclusive experiencesInterview tipsLeadership principlesWorking at AmazonFAQ×Software Engineer IJob ID: 3055226 | Twitch Interactive, Inc.Apply nowDESCRIPTIONIf you are interested in this position, please apply on Twitch's Career site https://www.twitch.tv/jobs/en/About Us:Twitch is the world’s biggest live streaming service, with global communities built around gaming, entertainment, music, sports, cooking, and more. It is where thousands of communities come together for whatever, every day.We’re about community, inside and out. You’ll find coworkers who are eager to team up, collaborate, and smash (or elegantly solve) problems together. We’re on a quest to empower live communities, so if this sounds good to you, see what we’re up to on LinkedIn and X,  and discover the projec

In [15]:
import re
import json

def extract_json_from_string(text):

    # Regex to find text between ```json and ```, using re.DOTALL to match newlines
    match = re.search(r'```json\n(.*?)```', text, re.DOTALL)
    if match:
        json_string = match.group(1).strip()
        try:
            return json.loads(json_string)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    return None

In [43]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON NO PREAMBLE :    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':content1})
jason_res = extract_json_from_string(res.content)
print(jason_res)

{'role': 'Software Engineer I', 'experience': '1+ Years experience developing software in a professional environment', 'skills': {'required': 'Proficient coding skills in modern languages and frameworks', 'preferred': 'Experience with languages/frameworks: Golang, Typescript/React; Experience building software and services using AWS technologies such as ECS, DynamoDB, Lambda, SQS, and Step Functions'}, 'description': 'The Twitch Ads team manages a comprehensive advertising ecosystem that balances revenue generation with user and advertiser satisfaction. We deliver promotional content through multiple channels while maintaining seamless experiences for viewers, creators, and advertisers. The role involves designing, developing, launching, and operating interactive experiences and products, building distributed applications at scale, and collaborating with various teams to deliver solutions. The position is based in San Francisco, CA, and offers perks such as medical, dental, vision & di

In [44]:
type(jason_res)

dict

In [23]:
from urllib.request import urlretrieve

In [24]:
urlretrieve('https://raw.githubusercontent.com/codebasics/project-genai-cold-email-generator/refs/heads/main/my_portfolio.csv', './dataset/my_portfolio.csv')

('./dataset/my_portfolio.csv', <http.client.HTTPMessage at 0x22e694f43e0>)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./dataset/my_portfolio.csv')

In [7]:
df1 = pd.read_csv('./app/resources/my_portfolio.csv')
df1

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [4]:
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [29]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [8]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

NameError: name 'collection' is not defined

In [39]:
def eliminate_think_tags(text):
    """
    Removes the <think>...</think> block from a given string.

    Args:
        text (str): The input string containing the <think> tags.

    Returns:
        str: The string with the <think> block removed.
    """
    # The regex pattern matches the <think> tag, any characters (including newlines)
    # in between, and the </think> tag. The re.DOTALL flag ensures that '.'
    # matches newline characters.
    pattern = re.compile(r'<think>.*?</think>', re.DOTALL)
    
    # Use re.sub() to replace the matched pattern with an empty string.
    cleaned_text = pattern.sub('', text)
    
    # Strip any leading/trailing whitespace left after the removal.
    return cleaned_text.strip()

In [45]:
job = jason_res

In [46]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(eliminate_think_tags( res.content))

Subject: Expert Software Engineering Solutions for Twitch Ads Team  

Dear Hiring Manager,  

I hope this message finds you well. My name is Mohan, and I’m a Business Development Executive at AtliQ, an AI & Software Consulting company. We specialize in empowering businesses like Twitch with tailored software solutions that drive scalability, process optimization, and cost reduction.  

With your team’s focus on building distributed applications at scale and delivering seamless advertising experiences, I believe AtliQ’s expertise aligns perfectly with your needs. Our team excels in modern languages and frameworks, including Golang and Typescript/React, and has extensive experience with AWS technologies such as ECS, DynamoDB, Lambda, SQS, and Step Functions. We’ve helped numerous enterprises design, develop, and operate large-scale systems, ensuring high performance and reliability.  

To give you a glimpse of our capabilities, here are some relevant portfolio pieces:  
- [Vue.js Portfol